Test slab_model.py
==================

Jinbo Wang

Jinbo.Wang@jpl.nasa.gov

last update: 12/31/2021

copied from full_wacm-scenario.ipynb

# WaCM was generated from the original

Highpass filtering and detides will be performed on the simulated WaCM data.

In [1]:
import slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils,mysignal
from scipy.io import loadmat
import pandas as pd


xrod=xr.open_dataset
t0,t1='2007-09-20','2007-09-29'

#load data station papa
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
f0=utils.lat2f(lat0)
tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')


#fn='../data/from_Hong/KEO2.mat'
#lat0=32.3
#tt=pd.date_range('2007-09-26 11:00:00',periods=8255,freq='1h')
#t0,t1='2007-11-20','2007-12-20'
#t0,t1='2007-12-13','2008-01-30'

d=loadmat(fn)

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )#.sel(time=slice(t0,t1))
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )#.sel(time=slice(t0,t1))

orbit='500_1000'
orbit='700_1800'
#orbit='hourly'
#orbit='1h'

d700=slab_model.synthetic_wacm(uobs,vobs,uwobs,vwobs,lat0,
                               cutoff_truth=1/3,cutoff_wacm=1/3,
                               orbit=orbit)


FileNotFoundError: [Errno 2] No such file or directory: '../data/from_Hong/PaPa2.mat'

In [ ]:
fig,ax=plt.subplots(2,1,figsize=(16,10))

dn=d700.u_truth_high_notide_nio
cc=(((dn)**2).resample(time='5D').mean()**0.5)
cc.plot(ax=ax[0],color='k',lw=1,label='true NIO')

ax[1].plot(np.sort(cc)[::-1].cumsum())

### Loop through the full year

In [ ]:
time=d700.u_wacm_total.time.values
u,v,H,c=[],[],[],[]

wacm_sampling=slab_model.load_wacm_period(lat0,orbit)*3600

ndays=10 #days
nn=int((ndays*86400)//wacm_sampling)

freq=8 #days
fig,ax=plt.subplots(figsize=(20,5))
has_tides=False

j0=60
for i in range(1):
    for j in np.arange(j0,time.size,nn)[:1]:
        
        t00=time[j-1]
        t11=time[j+nn+2] 
        uwacm=d700.u_truth_total.sel(time=slice(t00,t11))
        t0=uwacm.time.values[0]
        t1=uwacm.time.values[-1]
        #t0,t1=np.datetime64('2007-09-21'),np.datetime64('2007-09-29')

        up,vp,utid, vtid, param=d700.get_wacm_nio(t0,t1,has_tides=has_tides,use_hourly_wind=False)
        
        slab_model.NIO_bandpass(up,f0).plot()
        (up+utid).plot()
        
        up   = up.sel( time=slice(time[j],time[j+nn]-np.timedelta64(1,'h') ) )
        vp   = vp.sel( time=slice(time[j],time[j+nn]-np.timedelta64(1,'h') ) )
        if has_tides:
            utid = utid.sel( time=slice(time[j],time[j+nn]-np.timedelta64(1,'h') ) )
            vtid = vtid.sel( time=slice(time[j],time[j+nn]-np.timedelta64(1,'h') ) )
          
        d700.u_truth_high_notide_nio.sel(time=slice(t0,t1)).plot(color='gray',lw=2,label='true NIO')
        #d700.u_truth_high.sel(time=slice(t00,t11)).plot(color='k',linestyle='--',lw=2,label='total obs')

        if j==j0:    
            upp=up*1;vpp=vp*1
            if has_tides:
                utp=utid*1;vtp=vtid*1
        else:
            upp=xr.concat([upp,up*1],dim='time')
            vpp=xr.concat([vpp,vp*1],dim='time')
            if has_tides:
                utp=xr.concat([utp,utid*1],dim='time')
                vtp=xr.concat([vtp,vtid*1],dim='time')
        del up,vp,utid,vtid
        
    u.append(upp)
    v.append(vpp)
        
plt.legend()

In [ ]:
time=d700.u_wacm_total.time.values
u,v,H,c=[],[],[],[]
print(time.size)
nn=10 #days

freq=1 #days
fig,ax=plt.subplots(figsize=(30,10))

for i in range(1):
    print(i)
    for j in pd.date_range(time[20],periods=4,freq='%iD'%freq):
        t00=j-np.timedelta64(int(nn/2*24)-i*12,'h') 
        t11=j+np.timedelta64(int(nn/2*24)+i*12,'h') 
        uwacm=d700.u_truth_total.sel(time=slice(t00,t11))
        t0=uwacm.time.values[0]
        t1=uwacm.time.values[-1]
        newt=pd.date_range(j-np.timedelta64(freq*12,'h'), periods=freq*24, freq='h')
        
        #t0=time[j];t1=time[j+nn]
        up,vp,param=d700.get_wacm_nio(t0,t1,has_tides=True,use_hourly_wind=False)
        up.plot(ax=ax)
        try:    
            upp=xr.concat([upp,up.interp(time=newt)],dim='time')
            vpp=xr.concat([vpp,vp.interp(time=newt)],dim='time')
            print(t0,t1,param['cost'],param['H'])
            del up,vp
        except:
            upp=up.interp(time=newt)
            vpp=vp.interp(time=newt)
            print(t0,t1,param['cost'],param['H'])
        

    u.append(upp)
    v.append(vpp)
    del upp,vpp
d700.u_truth_high_notide.sel(time=slice(u[0].time.values[0],u[-1].time.values[-1])).plot(color='k',lw=2)
d700.u_truth_high_notide_nio.sel(time=slice(u[0].time.values[0],u[-1].time.values[-1])).plot(color='gray',lw=2)

In [ ]:
fig,ax=plt.subplots(figsize=(30,10))
d700.u_truth_high_notide.sel(time=slice(u[0].time.values[0],u[-1].time.values[-1])).plot(color='k',lw=3)
#d700.u_truth_high_notide_nio.sel(time=slice(u[0].time.values[0],u[-1].time.values[-1])).plot(color='r',lw=1)
for uu in u:
    c=mysignal.filter_butter(uu.values,[0.96*f0,1.04*f0],2*np.pi/3600,'bandpass')
    uu.plot()
    u0=uu.copy()
    u0.data[:]=c[:]
    #u0.plot(color='g')


In [ ]:
#loop through every ten days


ttend=d700.u_wacm_total[-1].time.data

total_days=uobs.size//24

for t00 in d700.u_wacm_total[:4].time.values:
    u_wacm_nio=uobs*0
    v_wacm_nio=vobs*0
    H_wacm=uobs*0
    c_wacm=uobs*0
    #print(i,np.datetime64('now'))
    #for ii in range(i,8541-240,240)[:5]: 
    print(t00)
    print(pd.date_range(t00,'2008-05-28T21:20:00.000000000',freq='6D'))
    for it0 in np.range(0,total_days,6*24):
        t0=t00+np.timedelta64(it0,'h')
        t1=t0+np.timedelta64(it0+6*24,'h')
        t_out=u_wacm_nio.sel(time=slice(t0,t1)).data[:]
        up,vp,param=d700.get_wacm_nio(t0,t1,use_hourly_wind=True)
        u_wacm_nio.sel(time=slice(t0,t1)).data[:]=up.interp(time=t_out).data[:]
        v_wacm_nio.sel(time=slice(t0,t1)).data[:]=vp.interp(time=t_out).data[:]
        #H_wacm[ii:ii+240].data[:]=param['H']
        #c_wacm[ii:ii+240].data[:]=param['c']
    u.append(u_wacm_nio)
    v.append(v_wacm_nio)
    #H.append(H_wacm)
    #c.append(c_wacm)
    

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(10,10))
d700.u_truth_high_notide_nio.sel(time=slice('2007-09-20 00:00:00',t1)).plot(ax=ax[0],color='k',lw=3)

for uu in u:
    uu.plot(ax=ax[0])


In [ ]:
uu=xr.DataArray(np.zeros((24,u[0].size)), dims=('case','time'),
                coords={'case':range(24),'time':u[0].time.values})
vv=xr.DataArray(np.zeros((24,u[0].size)), dims=('case','time'),
                coords={'case':range(24),'time':u[0].time.values})

for i in range(4):
    uu[i,:]=u[i].interp(time=u[0].time.values)
    vv[i,:]=v[i].interp(time=u[0].time.values)

fig,ax=plt.subplots(3,1,figsize=(10,10))

tt0,tt1='2007-06-20','2007-06-29'
for i in range(4):
    #uu[i,:].sel(time=slice(tt0,tt1)).plot(ax=ax[0],color='gray')
    u[i].sel(time=slice(tt0,tt1)).plot(ax=ax[0])
    d700.u_truth_high_notide_nio.sel(time=slice(tt0,tt1)).plot(ax=ax[0],color='gray',lw=3)
    
uu=d700.u_truth_high_notide_nio
vv=d700.v_truth_high_notide_nio


((uu**2+vv**2)**0.5).resample(time='1D').mean().plot(color='red',lw=1)

In [ ]:
((u_wacm_nio**2+v_wacm_nio**2)**0.5).resample(time='1D').mean().plot(label='wacm')
((d700.u_truth_high_notide_nio**2+d700.v_truth_high_notide_nio**2)**0.5).resample(time='1D').mean().plot(label='true')
plt.legend()